In [1]:
import jax
from jax.sharding import Mesh

from deeprte.configs import default
from deeprte.input_pipeline import input_pipeline_interface
from deeprte.train_lib import utils
from deeprte.train_lib.multihost_dataloading import prefetch_to_device

In [2]:
config = default.get_config()

In [3]:
device_array = utils.create_device_mesh(config)
global_mesh = Mesh(device_array, config.mesh_axes)
global_mesh

Mesh(device_ids=array([[[0],
        [1]]]), axis_names=('data', 'fsdp', 'tensor'))

In [4]:
(train_iter, eval_iter), data_sharding = input_pipeline_interface.create_data_iterator(
    config, global_mesh
)

if config.prefetch_to_device:
    train_iter = prefetch_to_device(train_iter, 4)

In [5]:
data_sharding

NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))

In [6]:
jax.tree.map(lambda x: (x.shape, x.sharding), next(train_iter))

{'boundary': ((8, 1920),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))),
 'boundary_coords': ((8, 1920, 4),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))),
 'boundary_scattering_kernel': ((8, 1920, 24),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))),
 'boundary_weights': ((8, 1920),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))),
 'phase_coords': ((8, 128, 4),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))),
 'position_coords': ((8, 1600, 2),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', 'fsdp', 'tensor'),))),
 'psi_label': ((8, 128),
  NamedSharding(mesh=Mesh('data': 1, 'fsdp': 2, 'tensor': 1), spec=PartitionSpec(('data', '